# Backend Verification for the iHP Open MPW Shuttles

[D. Mitch Bailey](https://www.linkedin.com/in/mitch-bailey-cvc/), [ShuhariSystem](https://www.shuharisystem.com/)

# Overview
This notebook performs backend LVS (with [klayout](https://github.com/KLayout/klayout) or [magic](https://github.com/RTimothyEdwards/magic)/[netgen](https://github.com/RTimothyEdwards/netgen)), soft-connection checks (with magic/netgen) and/or reliability verfication (with [CVC-RV](https://github.com/d-m-bailey/cvc)) with open source EDA software and systems.

# wafer.space Dec. 2025 Tinytapeout Data
The Tinytapeout submission to gf180 wafer.space shuttle for Dec. 2025 listed below are located [here](https://github.com/TinyTapeout/tinytapeout-gf-0p2.git).

Create a file with the initial environment variables.


In [32]:
%%writefile /content/env
export LOCAL_INSTALL=/content/local
export PATH=$PATH:$LOCAL_INSTALL/bin
export PDK_ROOT=/content/pdks
export PDK=gf180mcuD
export PDKPATH=$PDK_ROOT/$PDK
# ddb601a4a4473163e1ed6df416b885df18b4ac03 (2025.01.08)
# export PDK_COMMIT=ddb601a4a4473163e1ed6df416b885df18b4ac03
# cb7daaa8901016cf7c5d272dfa322c41f024931f (2025.07.18)
export PDK_COMMIT=486fb8633c3d95ae475a5ccd68d0066c25919bb5
export MAGIC_VERSION=8.3.538
export NETGEN_VERSION=1.5.295
export KLAYOUT_DISTRO=Ubuntu-22
export KLAYOUT_DEB=klayout_0.30.3-1_amd64.deb
export EXTRA_CHECK_VERSION=chipathon2025
export CVC_VERSION=master
export DATA_ROOT=/content/data
export UPRJ_ROOT=$DATA_ROOT
export CHECK_ROOT=/content/check_data
export CHECK_VERSION=layout-update-file
export LVS_ROOT=/root/extra_be_checks
export MPW=WS_2025
export SINGLE_THREAD_EXTRACTION=yes
export CARAVEL_ROOT=
if [[ -f $DATA_ROOT/project_env ]]; then
  cat $DATA_ROOT/project_env
  source $DATA_ROOT/project_env
fi


Overwriting /content/env


# Program installation.
Only needs to be executed once.

This step sets up the pdk and installs magic, klayout, netgen and cvc_rv.

Runtime: 3-4 minutes

Note: klayout LVS rule files are modified as follows:

Note: The magic tech file `gf180mcuD.tech` from `extra_be_checks/tech/gf180mcuD` replaces the pdk tech file and contains the following modifications:

Note: netgen setup file `gf180mcuD_setup.tcl` is modified as follows:



In [45]:
%%shell
cd
cat /content/env
source /content/env

lsb_release -a 2>/dev/null

sudo apt update

if ! command -v ciel; then
  echo "==> Installing ciel..."
  pip install ciel
fi
rm -rf $PDK_ROOT
ciel enable --pdk $PDK $PDK_COMMIT
echo "==> Using pdk $PDK commit $PDK_COMMIT (patched)
"

echo "==> Downloading extra_be_checks $EXTRA_CHECK_VERSION"
rm -rf extra_be_checks
git clone https://github.com/d-m-bailey/extra_be_checks.git -b $EXTRA_CHECK_VERSION
# Exchange npn13g2 l/w params
echo "==> Using extra_be_checks commit $(cd extra_be_checks; git rev-parse HEAD)
"

if ! command -v netgen; then
  echo "==> Downloading and installing netgen $NETGEN_VERSION"
  git clone https://github.com/RTimothyEdwards/netgen.git --depth=1 -b $NETGEN_VERSION
  cd netgen
  ./configure --prefix=$LOCAL_INSTALL
  make
  make install
  cd
fi
echo "==> Using netgen version $(netgen -batch | awk '/Netgen/ {print $2}')
"

if ! command -v magic; then
  echo "==> Downloading and installing magic $MAGIC_VERSION"
  git clone https://github.com/RTimothyEdwards/magic.git --depth=1 -b $MAGIC_VERSION
  cd magic
  ./configure --prefix=$LOCAL_INSTALL
  make
  make install
  cd
fi
echo "==> Using magic version $(magic -dnull -noc --version)
"

if ! command -v klayout; then
  echo "==> Downloading and installing klayout $KLAYOUT_DEB for $KLAYOUT_DISTRO"
  wget -P /tmp https://www.klayout.org/downloads/$KLAYOUT_DISTRO/$KLAYOUT_DEB
  sudo dpkg -i /tmp/$KLAYOUT_DEB
  sudo apt --fix-broken install
  pip install docopt
  pip install klayout
  cd
fi
echo "==> Using $(klayout -v)
"

if ! command -v cvc_rv; then
  echo "==> Downloading and installing cvc_rv $CVC_VERSION"
  sudo apt install autopoint bison flex
  git clone https://github.com/d-m-bailey/cvc --depth=1 -b $CVC_VERSION
  cd cvc
  autoreconf -vif
  ./configure --prefix=$LOCAL_INSTALL --disable-nls
  make
  make install
  cd
fi
echo "==> Using $(cvc_rv -v)
"

echo "#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~"
echo "pdk $PDK commit $PDK_COMMIT (patched)"
echo "extra_be_checks commit $(cd extra_be_checks; git rev-parse HEAD)"
echo "netgen version $(netgen -batch | awk '/Netgen/ {print $2}')"
echo "magic version $(magic -dnull -noc --version)"
echo "$(klayout -v)"
echo "$(cvc_rv -v)"
echo "#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~"

export LOCAL_INSTALL=/content/local
export PATH=$PATH:$LOCAL_INSTALL/bin
export PDK_ROOT=/content/pdks
export PDK=gf180mcuD
export PDKPATH=$PDK_ROOT/$PDK
# ddb601a4a4473163e1ed6df416b885df18b4ac03 (2025.01.08)
# export PDK_COMMIT=ddb601a4a4473163e1ed6df416b885df18b4ac03
# cb7daaa8901016cf7c5d272dfa322c41f024931f (2025.07.18)
export PDK_COMMIT=486fb8633c3d95ae475a5ccd68d0066c25919bb5
export MAGIC_VERSION=8.3.538
export NETGEN_VERSION=1.5.295
export KLAYOUT_DISTRO=Ubuntu-22
export KLAYOUT_DEB=klayout_0.30.3-1_amd64.deb
export EXTRA_CHECK_VERSION=chipathon2025
export CVC_VERSION=master
export DATA_ROOT=/content/data
export UPRJ_ROOT=$DATA_ROOT
export CHECK_ROOT=/content/check_data
export CHECK_VERSION=layout-update-file
export LVS_ROOT=/root/extra_be_checks
export MPW=WS_2025
export SINGLE_THREAD_EXTRACTION=yes
export CARAVEL_ROOT=
if [[ -f $DATA_ROOT/project_env ]]; then
  cat $DATA_ROOT/project_env
  source $DATA_ROOT/project_env
fi
export LAYOUT_TOP=tt_gf_wrapper
export ORIGINAL_DATA=$

# Clone the design repo.

Runtime: 1-2 minutes.

In [11]:
%%shell
cat /content/env
source /content/env
rm -rf $DATA_ROOT $CHECK_ROOT
git clone https://github.com/d-m-bailey/ihp-mpw-be.git --depth=1 -b $CHECK_VERSION $CHECK_ROOT
git clone https://github.com/TinyTapeout/tinytapeout-gf-0p2.git --depth=1 $DATA_ROOT
wget -P $DATA_ROOT https://github.com/TinyTapeout/tinytapeout-gf-0p2/releases/download/tapeout-ws2512/tt_gf_wrapper.oas


export LOCAL_INSTALL=/content/local
export PATH=$PATH:$LOCAL_INSTALL/bin
export PDK_ROOT=/content/pdks
export PDK=gf180mcuD
export PDKPATH=$PDK_ROOT/$PDK
# ddb601a4a4473163e1ed6df416b885df18b4ac03 (2025.01.08)
# export PDK_COMMIT=ddb601a4a4473163e1ed6df416b885df18b4ac03
# cb7daaa8901016cf7c5d272dfa322c41f024931f (2025.07.18)
export PDK_COMMIT=486fb8633c3d95ae475a5ccd68d0066c25919bb5
export MAGIC_VERSION=8.3.538
export NETGEN_VERSION=1.5.295
export KLAYOUT_DISTRO=Ubuntu-22
export KLAYOUT_DEB=klayout_0.30.3-1_amd64.deb
export EXTRA_CHECK_VERSION=chipathon2025
export CVC_VERSION=master
export DATA_ROOT=/content/data
export CHECK_ROOT=/content/check_data
export CHECK_VERSION=layout-update-file
export LVS_ROOT=/root/extra_be_checks
export MPW=WS_2025
if [[ -f $DATA_ROOT/project_env ]]; then
  cat $DATA_ROOT/project_env
  source $DATA_ROOT/project_env
fi
export LAYOUT_TOP=gf_tt_wrapper
export ORIGINAL_DATA=$DATA_ROOT/$LAYOUT_TOP.oas
export LAYOUT_DATA=$DATA_ROOT/$LAYOUT_TOP.gds.gz
export WOR

# ※ Run cells before this cell for program installation and general setup.<br>Then choose one setup from the 17 designs below.<br>Finally, run the checks and examine the results

General results:
1. When the layout has cells that have the same name as primitive devices, netgen segfaults. <br>➡︎ Flatglob all layout cells with primitive device names.

# 1. gf_tt_wrapper

Set the project, top cell and gds file names.

In [34]:
%%shell
source /content/env
cat > $DATA_ROOT/project_env <<'EOF'
export LAYOUT_TOP=tt_gf_wrapper
export ORIGINAL_DATA=$DATA_ROOT/$LAYOUT_TOP.oas
export LAYOUT_DATA=$DATA_ROOT/$LAYOUT_TOP.gds
export WORK_ROOT=$DATA_ROOT/work/$LAYOUT_TOP
# original data
# modified data
EOF

cat $DATA_ROOT/project_env
source $DATA_ROOT/project_env
rm -f $DATA_ROOT/work
rm -rf $WORK_ROOT
mkdir -p $WORK_ROOT

cat > $DATA_ROOT/empty.json <<'EOF'
{ "cells": {} }
EOF

export LAYOUT_TOP=tt_gf_wrapper
export ORIGINAL_DATA=$DATA_ROOT/$LAYOUT_TOP.oas
export LAYOUT_DATA=$DATA_ROOT/$LAYOUT_TOP.gds.gz
export WORK_ROOT=$DATA_ROOT/work/$LAYOUT_TOP
# original data
# modified data
export LAYOUT_TOP=tt_gf_wrapper
export ORIGINAL_DATA=$DATA_ROOT/$LAYOUT_TOP.oas
export LAYOUT_DATA=$DATA_ROOT/$LAYOUT_TOP.gds
export WORK_ROOT=$DATA_ROOT/work/$LAYOUT_TOP
# original data
# modified data
rm: cannot remove '/content/data/work': Is a directory


Create `lvs_config.json` and update the `LVS_SPICE_FILES` and `LVS_VERILOG_FILES` for every design.

Update the other parameters as needed.

In [52]:
%%shell
cat /content/env
source /content/env
cat > $WORK_ROOT/lvs_config.json <<EOF
{
        "STD_CELL_LIBRARY": "gf180mcu_fd_sc_mcu7t5v0",
        "INCLUDE_CONFIGS": [
                "$LVS_ROOT/tech/$PDK/lvs_config.base.json"
        ],
        "TOP_SOURCE": "tt_gf_wrapper",
        "TOP_LAYOUT": "\$TOP_SOURCE",
        "EXTRACT_FLATGLOB": [
                ""
        ],
        "EXTRACT_ABSTRACT": [
                "gf180mcu_fd_io__asig_5p0",
                "gf180mcu_fd_io__bi_24t",
                "gf180mcu_fd_io__cor",
                "gf180mcu_fd_io__dvdd",
                "gf180mcu_fd_io__dvss",
                "gf180mcu_fd_io__fill*",
                "gf180mcu_fd_io__in_c",
                "gf180mcu_fd_io__in_s",
                "*fill_cell"
        ],
        "LVS_FLATTEN": [
        ],
        "LVS_NOFLATTEN": [
                ""
        ],
        "LVS_IGNORE": [
                "*fill_cell"
        ],
        "LVS_SPICE_FILES": [
                ""
        ],
        "LVS_VERILOG_FILES": [
        ],
        "LAYOUT_FILE": "$LAYOUT_DATA"
}
EOF

export LOCAL_INSTALL=/content/local
export PATH=$PATH:$LOCAL_INSTALL/bin
export PDK_ROOT=/content/pdks
export PDK=gf180mcuD
export PDKPATH=$PDK_ROOT/$PDK
# ddb601a4a4473163e1ed6df416b885df18b4ac03 (2025.01.08)
# export PDK_COMMIT=ddb601a4a4473163e1ed6df416b885df18b4ac03
# cb7daaa8901016cf7c5d272dfa322c41f024931f (2025.07.18)
export PDK_COMMIT=486fb8633c3d95ae475a5ccd68d0066c25919bb5
export MAGIC_VERSION=8.3.538
export NETGEN_VERSION=1.5.295
export KLAYOUT_DISTRO=Ubuntu-22
export KLAYOUT_DEB=klayout_0.30.3-1_amd64.deb
export EXTRA_CHECK_VERSION=chipathon2025
export CVC_VERSION=master
export DATA_ROOT=/content/data
export UPRJ_ROOT=$DATA_ROOT
export CHECK_ROOT=/content/check_data
export CHECK_VERSION=layout-update-file
export LVS_ROOT=/root/extra_be_checks
export MPW=WS_2025
export SINGLE_THREAD_EXTRACTION=yes
export CARAVEL_ROOT=
if [[ -f $DATA_ROOT/project_env ]]; then
  cat $DATA_ROOT/project_env
  source $DATA_ROOT/project_env
fi
export LAYOUT_TOP=tt_gf_wrapper
export ORIGINAL_DATA=$

Create the cvcrc setup file and power file.

In [40]:
%%shell
cat /content/env
source /content/env
cat > $WORK_ROOT/cvcrc <<EOF
CVC_TOP = $LAYOUT_TOP
CVC_NETLIST = $WORK_ROOT/ext/$LAYOUT_TOP.cdl.gz
CVC_MODEL_FILE = $LVS_ROOT/tech/ihp-sg13g2/cvc.models
CVC_POWER_FILE = $WORK_ROOT/cvc.power.$LAYOUT_TOP
CVC_REPORT_FILE = $WORK_ROOT/cvc.log
EOF

cat > $WORK_ROOT/cvc.power.$PROJECT <<EOF
VGND power 0.0
VPWR power 3.3
EOF

export LOCAL_INSTALL=/content/local
export PATH=$PATH:$LOCAL_INSTALL/bin
export PDK_ROOT=/content/pdks
export PDK=gf180mcuD
export PDKPATH=$PDK_ROOT/$PDK
# ddb601a4a4473163e1ed6df416b885df18b4ac03 (2025.01.08)
# export PDK_COMMIT=ddb601a4a4473163e1ed6df416b885df18b4ac03
# cb7daaa8901016cf7c5d272dfa322c41f024931f (2025.07.18)
export PDK_COMMIT=486fb8633c3d95ae475a5ccd68d0066c25919bb5
export MAGIC_VERSION=8.3.538
export NETGEN_VERSION=1.5.295
export KLAYOUT_DISTRO=Ubuntu-22
export KLAYOUT_DEB=klayout_0.30.3-1_amd64.deb
export EXTRA_CHECK_VERSION=chipathon2025
export CVC_VERSION=master
export DATA_ROOT=/content/data
export UPRJ_ROOT=$DATA_ROOT
export CHECK_ROOT=/content/check_data
export CHECK_VERSION=layout-update-file
export LVS_ROOT=/root/extra_be_checks
export MPW=WS_2025
export SINGLE_THREAD_EXTRACTION=yes
export CARAVEL_ROOT=
if [[ -f $DATA_ROOT/project_env ]]; then
  cat $DATA_ROOT/project_env
  source $DATA_ROOT/project_env
fi
export LAYOUT_TOP=tt_gf_wrapper
export ORIGINAL_DATA=$

Create the klayout CDL file from the magic source netlist.

In [ ]:
%%shell
source /content/env
sed -e '/^X[CDLMQR]/s/.//' \
    -e "s/\.subckt $PROJECT/.subckt $LAYOUT_TOP/I" $SPICE_FILE > $CDL_FILE


Modify the layout file to pass LVS. (see explanation above.)

In [41]:
%%shell
source /content/env
$CHECK_ROOT/scripts/apply_json_to_layout.py \
  --input_file=$ORIGINAL_DATA \
  --json_file=$DATA_ROOT/empty.json \
  --output_file=$LAYOUT_DATA

export LAYOUT_TOP=tt_gf_wrapper
export ORIGINAL_DATA=$DATA_ROOT/$LAYOUT_TOP.oas
export LAYOUT_DATA=$DATA_ROOT/$LAYOUT_TOP.gds
export WORK_ROOT=$DATA_ROOT/work/$LAYOUT_TOP
# original data
# modified data


---
# ※ Run BE checks


In [ ]:
%%shell
cat /content/env
source /content/env
# run_be_checks includes magic/netgen soft connection and LVS, and CVC-RV
$LVS_ROOT/run_be_checks $WORK_ROOT/lvs_config.json

#python3 $PDK_ROOT/$PDK/libs.tech/klayout/tech/lvs/run_lvs.py --layout=$LAYOUT_DATA \
#  --netlist=$CDL_FILE \
#  --run_dir=$WORK_ROOT/klayout \
#  --topcell=$LAYOUT_TOP \
#  --run_mode=deep \
#  --spice_comments \
#  --no_simplify \
#  --combine_devices \
#  --top_lvl_pins
#
#LVS_BASENAME=$( echo $LAYOUT_DATA | sed -e 's,.*/,,' -e 's,\..*,,' )
#$LVS_ROOT/check_klayout_ports $WORK_ROOT/klayout/$LVS_BASENAME.lvsdb > $WORK_ROOT/klayout/port_check.log
#if [[ -e $WORK_ROOT/klayout/port_check.log ]]; then
#  if [[ -s $WORK_ROOT/klayout/port_check.log ]]; then
#    echo "* Klayout ports do not match *"
#    echo "layout     source"
#    awk '{printf "%-10s %s\n", $1, $2}' $WORK_ROOT/klayout/port_check.log
#  else
#    echo "Klayout ports match"
#  fi
#fi

export LOCAL_INSTALL=/content/local
export PATH=$PATH:$LOCAL_INSTALL/bin
export PDK_ROOT=/content/pdks
export PDK=gf180mcuD
export PDKPATH=$PDK_ROOT/$PDK
# ddb601a4a4473163e1ed6df416b885df18b4ac03 (2025.01.08)
# export PDK_COMMIT=ddb601a4a4473163e1ed6df416b885df18b4ac03
# cb7daaa8901016cf7c5d272dfa322c41f024931f (2025.07.18)
export PDK_COMMIT=486fb8633c3d95ae475a5ccd68d0066c25919bb5
export MAGIC_VERSION=8.3.538
export NETGEN_VERSION=1.5.295
export KLAYOUT_DISTRO=Ubuntu-22
export KLAYOUT_DEB=klayout_0.30.3-1_amd64.deb
export EXTRA_CHECK_VERSION=chipathon2025
export CVC_VERSION=master
export DATA_ROOT=/content/data
export UPRJ_ROOT=$DATA_ROOT
export CHECK_ROOT=/content/check_data
export CHECK_VERSION=layout-update-file
export LVS_ROOT=/root/extra_be_checks
export MPW=WS_2025
export SINGLE_THREAD_EXTRACTION=yes
export CARAVEL_ROOT=
if [[ -f $DATA_ROOT/project_env ]]; then
  cat $DATA_ROOT/project_env
  source $DATA_ROOT/project_env
fi
export LAYOUT_TOP=tt_gf_wrapper
export ORIGINAL_DATA=$

# View verification logs

In [ ]:
from google.colab import files

files.view('/content/data/work/soft.log')
files.view('/content/data/work/lvs.log')
files.view('/content/data/work/cvc.log')

# View soft verification report

In [ ]:
from google.colab import files

files.view('/content/data/work/soft.report')

# View LVS report

In [ ]:
from google.colab import files

files.view('/content/data/work/lvs.report')

# View CVC report

In [ ]:
from google.colab import files

files.view('/content/data/work/cvc.error')